In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import (mean, std)
from sklearn.datasets import make_classification
from sklearn.model_selection import (cross_val_score, RepeatedStratifiedKFold)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import (StackingClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, 
                             AdaBoostClassifier)
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from numpy import where
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
%matplotlib inline

In [ ]:
test = pd.read_csv('test_set_features.csv')
test['label'] = 'test'
train = pd.read_csv('training_set_features.csv')
train['label'] = 'train'
labels = pd.read_csv('training_set_labels.csv')
h1n1 = labels['h1n1_vaccine']
seasonal = labels['seasonal_vaccine']
combine = pd.concat([train, test], axis = 0)
ID = test['respondent_id']
combine = combine.drop(['respondent_id'], axis = 1)
full_train = pd.concat([labels, train], axis = 1)

In [ ]:
combine_cols = combine.columns
labels_cols = labels.columns

In [ ]:
#check column names
print(combine_cols)
print(labels_cols)

In [ ]:
#check the shape of the dataframes
print(train.shape)
print(test.shape)
print(labels.shape)
print(combine.shape)

In [ ]:
#look at the response variables
labels.describe()

In [ ]:
#26707 total rows
#21.2% got h1n1_vaccine
#46.6% got seasonal vaccine

In [ ]:
combine.describe()

In [ ]:
combine.dtypes

In [ ]:
all_data_na = (combine.isnull().sum()/len(combine))
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:40]
missing_data = pd.DataFrame({'Missing Ratio':all_data_na})
missing_data.head(40)

In [ ]:
#looks like employment occupation and employment industry is code for not in work force where blank.
combine['employment_industry'] = combine['employment_industry'].fillna('abcde')
combine['employment_occupation'] = combine['employment_occupation'].fillna('fghij')

In [ ]:
#split columns
combine[["msa", "city"]] = (  # Create two new features
    combine["census_msa"]           # from the Policy feature
    .str                         # through the string accessor
    .split(",", expand=True)     # by splitting on " "
                                 # and expanding the result into separate columns
)

In [ ]:
combine['race_and_sex'] = combine['race'] + "_" + combine['sex']

In [ ]:
label_enc = combine.select_dtypes(include = 'object').columns
print(label_enc)


In [ ]:
# process columns, apply LabelEncoder to categorical features
from sklearn.preprocessing import LabelEncoder
for c in label_enc:
    lbl = LabelEncoder() 
    lbl.fit(list(combine[c].values)) 
    combine[c] = lbl.transform(list(combine[c].values))

# shape        
print('Shape all_data: {}'.format(combine.shape))

In [ ]:
all_data_na = (combine.isnull().sum()/len(combine))
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:40]
missing_data = pd.DataFrame({'Missing Ratio':all_data_na})
missing_data.head(40)

In [ ]:
fig, ax = plt.subplots(figsize = (15,12))
plt.xticks(rotation='90')
sns.barplot(x=all_data_na.index, y=all_data_na)
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

In [ ]:
#anything with under 2% missing, just input the mode
cols = ('opinion_seas_sick_from_vacc', 'opinion_seas_risk' , 'opinion_seas_vacc_effective',
        'opinion_h1n1_vacc_effective','opinion_h1n1_sick_from_vacc', 'opinion_h1n1_risk', 'household_children',
        'household_adults', 'behavioral_avoidance', 'behavioral_touch_face', 'h1n1_knowledge', 'h1n1_concern',
        'behavioral_outside_home', 'behavioral_large_gatherings', 'behavioral_antiviral_meds', 'behavioral_wash_hands',
        'behavioral_face_mask')

for c in cols:
    combine[c] = combine[c].fillna(combine[c].mode()[0])
    

In [ ]:
all_data_na = (combine.isnull().sum()/len(combine))
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:40]

fig, ax = plt.subplots(figsize = (15,12))
plt.xticks(rotation='90')
sns.barplot(x=all_data_na.index, y=all_data_na)
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

In [ ]:
corrmat = combine.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=0.9, square=True)

In [ ]:
cols = ['doctor_recc_seasonal', 'doctor_recc_h1n1', 'chronic_med_condition', 'child_under_6_months',
                           'health_worker', 'health_insurance']
before = []
for c in cols:
    w = len(combine[c][combine[c]==1])
    wo = len(combine[c][combine[c]==0])
    pct_w = w / (w + wo)
    before.append(pct_w)
    print('percentage with', c, pct_w)

In [ ]:
#build model to fill values
# Oversample and plot imbalanced dataset with SMOTE
roc = []

for index, value in enumerate(cols):
    cols = ['health_insurance', 'doctor_recc_seasonal', 'doctor_recc_h1n1', 'chronic_med_condition', 'child_under_6_months',
                'health_worker']
   
    a = cols[index]
    cols.pop(index)
    X = combine.drop(columns = cols)
    W = X[X[a].notnull()]
    y = W[a]
    X = X.drop([a], axis = 1)
    W = W.drop([a], axis = 1)
    
    oversample = SMOTE()
    W, y = oversample.fit_resample(W,y)
    
    #model = DecisionTreeClassifier()
    model = GradientBoostingClassifier()
    # evaluate pipeline
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, W, y, scoring='roc_auc', cv=cv, n_jobs=-1)
    print('mean roc', a, mean(scores))
    #fit model
    model.fit(W,y)
    
    #make predictions
    combine['preds'] = model.predict(X)
    combine[a] = np.where(combine[a].isnull(), 
                                           combine['preds'], combine[a])
    combine = combine.drop(['preds'], axis = 1)

In [ ]:
combine.isnull().sum().any()

In [ ]:
#all NAs are gone

In [ ]:
#create baseline model with logistic regression
train = combine[combine['label'] == 1]
test = combine[combine['label'] == 0]
train = train.drop(['label'], axis = 1)
test = test.drop(['label'], axis = 1)
print(train.shape)
print(test.shape)

In [ ]:
#use grid search to find best parameters

""""
from sklearn.model_selection import GridSearchCV
# define the model with default hyperparameters
model = GradientBoostingClassifier()
X, y = train, h1n1
# define the grid of values to search
grid = dict()
grid['n_estimators'] = [10,50, 100]
grid['learning_rate'] = [0.0001, 0.001, 0.01]
grid['subsample'] = [0.5, 0.7, 1.0]
grid['max_depth'] = [3, 7, 9]

# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy')
# execute the grid search
grid_result = grid_search.fit(X, y)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    Best: 0.822331 using {'learning_rate': 0.01, 'max_depth': 9, 'n_estimators': 100, 'subsample': 0.5}

Best: 0.822331 using {'learning_rate': 0.01, 'max_depth': 9, 'n_estimators': 100, 'subsample': 0.5}
""""""

In [ ]:
# compare ensemble to each baseline classifier

  
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('lr', LogisticRegression()))
    level0.append(('rf', RandomForestClassifier()))
    level0.append(('boost', GradientBoostingClassifier()))
    level0.append(('et', ExtraTreesClassifier()))
    level0.append(('ada', AdaBoostClassifier()))
    # define meta learner model
    level1 = LogisticRegression()
    # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model
 
# get a list of models to evaluate
def get_models():
    models = dict()
    models['lr'] = LogisticRegression()
    models['rf'] = RandomForestClassifier()
    models['boost'] = GradientBoostingClassifier()
    models['et'] = ExtraTreesClassifier()
    models['ada'] = AdaBoostClassifier()
    models['stacking'] = get_stacking()
    return models
 
# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1, error_score='raise')
    return scores

#set data
X,y = (train, seasonal)
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
plt.boxplot(results, labels=names, showmeans=True)
plt.show()

In [ ]:
#set data
X = train
y = seasonal
#y = h1n1

In [ ]:
# define the base models
level0 = list()
level0.append(('lr', LogisticRegression(max_iter = 1000)))
level0.append(('rf', RandomForestClassifier()))
level0.append(('boost', GradientBoostingClassifier()))
#Best: 0.822331 using {'learning_rate': 0.01, 'max_depth': 9, 'n_estimators': 100, 'subsample': 0.5}
level0.append(('et', ExtraTreesClassifier()))
level0.append(('ada', AdaBoostClassifier()))
# define meta learner model
level1 = LogisticRegression()
# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
# fit the model on all available data
model.fit(X, y)
# make a prediction for one example
yhat = model.predict_proba(test)

In [ ]:
y_pred_h1n1_final = pd.DataFrame(yhat, columns = ['first_class', 'second_class'])

In [ ]:
y_pred_seasonal_final = pd.DataFrame(yhat, columns = ['first_class', 'second_class'])

In [ ]:
response = pd.concat([ID, y_pred_h1n1_final['second_class'],y_pred_seasonal_final['second_class']], axis = 1)
response.columns = ['respondent_id','h1n1_vaccine', 'seasonal_vaccine']
response.to_csv('predictions.csv', index=False)

In [ ]:
#.8408
#289 out of 1936. top 15%

In [ ]:
train = train.astype(int)
print(train.dtypes)

In [ ]:
#mutual information
# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = train.dtypes == int
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(train, seasonal, discrete_features)
mi_scores2 = make_mi_scores(train, h1n1, discrete_features)
print(mi_scores[::3])  # show seasonal scores
print(mi_scores[::3])  #show h1n1 scores

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores - Seasonal Vaccine")


plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores)

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores - H1N1 Vaccine")


plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores2)

In [ ]:
mi_scores[-9:]

In [ ]:
mi_scores2[-9:]

In [ ]:
#will keep all columns. There are some bad ones. But common sense tells me some of these columns matter. and the DF is not
#all that wide to start with. 35 total columns.

In [ ]:
#engineer new columns

In [ ]:
combine['family_size'] = combine['household_adults'] + combine['household_children']
combine['total_behaviors'] = combine['behavioral_antiviral_meds'] + combine['behavioral_avoidance'] + combine['behavioral_face_mask'] + combine['behavioral_wash_hands'] + combine['behavioral_large_gatherings'] + combine['behavioral_outside_home'] + combine['behavioral_touch_face']
combine['total_reccs'] = combine['doctor_recc_h1n1'] + combine['doctor_recc_seasonal']
combine['pro_vaxx'] =  combine['opinion_h1n1_risk'] + combine['opinion_h1n1_sick_from_vacc'] + combine['opinion_seas_risk'] + combine['opinion_seas_sick_from_vacc']
combine['works'] = combine['opinion_h1n1_vacc_effective'] + combine['opinion_seas_vacc_effective']


In [ ]:
#create baseline model with logistic regression
train_new = combine[combine['label'] == 1]
test_new = combine[combine['label'] == 0]
train_new = train_new.drop(['label'], axis = 1)
test_new = test_new.drop(['label'], axis = 1)

In [ ]:
X = train_new
y = seasonal
#y = h1n1

In [ ]:
print(train_new.shape)
print(test_new.shape)

In [ ]:
# define the base models
level0 = list()
level0.append(('lr', LogisticRegression(max_iter = 1000)))
level0.append(('rf', RandomForestClassifier()))
level0.append(('boost', GradientBoostingClassifier()))
#Best: 0.822331 using {'learning_rate': 0.01, 'max_depth': 9, 'n_estimators': 100, 'subsample': 0.5}
level0.append(('et', ExtraTreesClassifier()))
level0.append(('ada', AdaBoostClassifier()))
# define meta learner model
level1 = LogisticRegression()
# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
# fit the model on all available data
model.fit(X, y)
# make a prediction for one example
yhat = model.predict_proba(test_new)

In [ ]:
y_pred_seasonal_final = pd.DataFrame(yhat, columns = ['first_class', 'second_class'])

In [ ]:
y_pred_h1n1_final = pd.DataFrame(yhat, columns = ['first_class', 'second_class'])

In [ ]:
response = pd.concat([ID, y_pred_h1n1_final['second_class'],y_pred_seasonal_final['second_class']], axis = 1)
response.columns = ['respondent_id','h1n1_vaccine', 'seasonal_vaccine']
response.to_csv('predictions.csv', index=False)